Youtubers list:
- Benjamin Cowen - UCRvqjQPSeaWn-uEx-w0XOIg
- Brian Jung - UCQglaVhGOBI0BR5S6IJnQPg
- Coin bureau - UCqK_GSMbpiV8spgD3ZGloSw
- Craig Percoco - UCL-QLzGmf468WAL1U-9g0qA
- Coinsider - UCi7egjf0JDHuhznWugXq4hA
- Virtual Bacon - UCcrEA_xd9Ldf1C8DIJYdyyA
- Krypto Cove - UCAZ9PHafkZUFsGRsRPPWaxQ
- Crypto Insight - UCeho-pOR48pgRS5OQVifVCA
- Crypto Insight Desi - UCCbenQd7OteG1pG2Dn3tDMA
- CoinGecko - UC-OTgwOAI7KmP0eDAtqN3Ow
- Francis Dhun Uncensored - UC4RzRP-dlf-RrCl3iHQi77A
- JRNY Crypto - UC188KLMYLLGqVJZdYq7mYFw
- InvestAnswers - UClgJyzwGs-GyaNxUHcLZrkg
- Digital Asset News - UCJgHxpqfhWEEjYH9cLXqhIQ
- Crypto Banter - UCN9Nj4tjXbVTLYWN0EKly_Q
- Jerry Banfield Crypto - UCuP9POBXk191J_QWpR8a0TA
- Ivan on Tech - UCrYmtJBtLdtm2ov84ulV-yg
- Scott Melker - UCxIU1RFIdDpvA8VOITswQ1A

<br/>
Future:
- Crypto Mobi - Urdu

In [5]:
import pandas as pd
import pymongo
from pymongo import MongoClient
from googleapiclient.discovery import build
import pandas as pd
import seaborn as sns
from dotenv import load_dotenv, dotenv_values
import os
import json

from langchain.document_loaders import YoutubeLoader
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain

In [6]:
load_dotenv()
api_key = os.environ.get('YOUTUBE_API_KEY')

In [7]:
youtube = build("youtube", "v3", developerKey=api_key)

In [8]:
channel_ids = []

with open('channel_ids.json') as f:
    channel_data = json.load(f)

# Loop over each dictionary in the list
for channel in channel_data:
    # Access the 'id' key of each dictionary and append it to the list
    channel_ids.append(channel['id'])

def get_chann_stats(youtube, channel_ids):
    all_data=[]
    request = youtube.channels().list(
              part='snippet, contentDetails, statistics',
              id=','.join(channel_ids))
    response = request.execute()

    for i in range(len(response['items'])):
        data=dict(Channel_name=response['items'][i]['snippet']['title'],
                 Subscribers=response['items'][i]['statistics']['subscriberCount'],
                 Views=response['items'][i]['statistics']['viewCount'],
                 Total_videos=response['items'][i]['statistics']['videoCount'],
                 Playlist_id=response['items'][i]['contentDetails']['relatedPlaylists']['uploads'])
        all_data.append(data)
    
    return all_data


In [9]:
channel_Stats= get_chann_stats(youtube, channel_ids)
channel_data=pd.DataFrame(channel_Stats)
# channel_data=channel_data.sort_values(by=['Channel_name'])
channel_data

,Channel_name,Subscribers,Views,Total_videos,Playlist_id
0,Ivan on Tech,523000,27369106,961,UUrYmtJBtLdtm2ov84ulV-yg
1,Craig Percoco,427000,11982595,240,UUL-QLzGmf468WAL1U-9g0qA
2,Jerry Banfield Crypto,36300,2108580,242,UUuP9POBXk191J_QWpR8a0TA
3,VirtualBacon,322000,24482785,1130,UUcrEA_xd9Ldf1C8DIJYdyyA
4,InvestAnswers,505000,73864904,1154,UUlgJyzwGs-GyaNxUHcLZrkg
5,Crypto Banter,764000,121362013,2806,UUN9Nj4tjXbVTLYWN0EKly_Q
6,Benjamin Cowen,797000,111720013,2208,UURvqjQPSeaWn-uEx-w0XOIg
7,Scott Melker,146000,13179704,1493,UUxIU1RFIdDpvA8VOITswQ1A
8,CoinGecko,193000,6102499,668,UU-OTgwOAI7KmP0eDAtqN3Ow
9,Brian Jung,1780000,84295961,547,UUQglaVhGOBI0BR5S6IJnQPg


In [10]:
# https://www.youtube.com/watch?v=B_NQd2IeeFs
channel_data.to_csv("Crypto_youtubers_for_project.csv")

In [11]:
def get_latest_videos(youtube, channel_ids):
    all_data = []
    for channel_id in channel_ids:
        request = youtube.search().list(
            part="snippet",
            channelId=channel_id,
            maxResults=5,
            order="date"
        )
        response = request.execute()
        
        for item in response["items"]:
            data = {
                "Channel_name": response["items"][0]["snippet"]["channelTitle"],
                "Video_title": item["snippet"]["title"],
                "Video_url": f"https://www.youtube.com/watch?v={item['id']['videoId']}"
            }
            all_data.append(data)
    
    return pd.DataFrame(all_data)

# channel_stats = get_chann_stats(youtube, channel_ids)
latest_videos = get_latest_videos(youtube, channel_ids)


In [12]:
latest_videos.head()

,Channel_name,Video_title,Video_url
0,Benjamin Cowen,Bitcoin Dominance,https://www.youtube.com/watch?v=Px4tLjAQNV0
1,Benjamin Cowen,Bitcoin: The Beauty of Mathematics (Part 45),https://www.youtube.com/watch?v=qMIwh7UoPMo
2,Benjamin Cowen,Ethereum: Dubious Speculation,https://www.youtube.com/watch?v=doNwL_epMf0
3,Benjamin Cowen,Bitcoin: Dubious Speculation (The Movie),https://www.youtube.com/watch?v=eQFG_CwikjQ
4,Benjamin Cowen,Bitcoin On-Chain Analysis: MVRV Z-Score,https://www.youtube.com/watch?v=jxtm-Vlenyg


In [13]:
latest_videos.to_csv('latest_videos_of_each_youtuber.csv')

In [16]:
# loading video transcripts

loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=B_NQd2IeeFs", add_video_info=True)
result = loader.load()

In [24]:
transcript = result[0].page_content

"what are the best layer 2 token Investments going into this next bull market I've said I expect uh layer twos to be part of the Mania in the next when everyone gets crazy about crypto thousands of l2s launching is where the excitement's going to be and and then I'm going to make my next video talking about the scams to watch out for that also always comes with the crazy bull but in this video I want to talk about where the best places to put your money are now I've invested through eight bubbles I've done well but I'm not a trained financial advisor and I'm not corrupt enough for Golem Gensler so do your own research now before I even get into this I want to I want to explain my investment philosophy and I just I just tweeted uh my Twitter Dave LaVine zerom I just tweeted this because I saw a video with two of the biggest Tech Venture capitalists Brad gersner Bill Gurley made a video and and I tweeted that at 28 minutes and 20 seconds Bill Gurley says the largest companies have the hi

In [28]:
metadata = result[0].metadata

In [30]:
metadata['source'] = 'https://www.youtube.com/watch?v=' + metadata['source']
del metadata['description']
del metadata['thumbnail_url']
del metadata['length']

In [31]:
metadata

{'source': 'https://www.youtube.com/watch?v=B_NQd2IeeFs',
 'title': 'Layer two token portfolio recommendations - Part 5',
 'view_count': 489,
 'publish_date': '2024-01-28 00:00:00',
 'author': 'Dave Levine Dot Com'}